In [1]:
import pandas as pd
import numpy as np
import torch
import json
from normalizer import counter, subSample
from torch.utils.data import DataLoader

device = 'cpu' # Change this to cuda for GPU enabled computers

In [2]:
# All of our fields have names, but we need them in numbers, so define a dictionary to convert.
global magDict
magDict = {
    'TOTUSJH': 0,
    'TOTBSQ': 1,
    'TOTPOT': 2,
    'TOTUSJZ': 3,
    'ABSNJZH': 4,
    'SAVNCPP': 5,
    'USFLUX': 6,
    'TOTFZ': 7,
    'MEANPOT': 8,
    'EPSZ': 9,
    'SHRGT45': 10,
    'MEANSHR': 11,
    'MEANGAM': 12,
    'MEANGBT': 13,
    'MEANGBZ': 14,
    'MEANGBH': 15,
    'MEANJZH': 16,
    'TOTFY': 17,
    'MEANJZD': 18,
    'MEANALP': 19,
    'TOTFX': 20,
    'EPSY': 21,
    'EPSX': 22,
    'R_VALUE': 23,
    'RBZ_VALUE': 24,
    'RBT_VALUE': 25,
    'RBP_VALUE': 26,
    'FDIM': 27,
    'BZ_FDIM': 28,
    'BT_FDIM': 29,
    'BP_FDIM': 30,
    'PIL_LEN': 31,
    'XR_MAX': 32
}

In [3]:
# Get the data from the JSON file, then return it as a tensor of input data and a list of labels
def getDataFromJSON(path="data/train_partition1_data.json", device='cpu', earlyStop=-1):
    # path is the path to the files, device is where to store it (CUDA), earlyStop is how many lines to 
    # read if you don't want the entire file read.
    
    # Get the dictionary to assign names to numbers
    global magDict
    
    # This dataset is heavily skewed, so we need to get the number of each type of flare.
    # This also lets us get the number of lines in the file with a sum.
    # This function also ignores any lines with a value of NaN.
    weights = counter(path, earlyStop)
    lines = np.sum(weights)
    # Check when we want to stop - the end of the file or earlier.
    if earlyStop < 0: length = lines
    else: length = min(earlyStop, lines)
    
    # Get the file and open it. 
    file = open(path)    
    
    # Declare a tensor to hold the data, and a list to hold the labels.
    # Dimensions: 0: number of entries we want. 1: the 33 fields in the data. 2: the 60 observations in each field. 
    tnsr = torch.Tensor().new_empty((length, 33, 60), device=device)
    labels = []
    flares = {'X':0, 'M':1, 'C':2, 'B':3, 'Q':4}
        
    row = -1
    for line in file:
        if 'nan' in line or "NaN" in line:
            continue
        # Load the line as a dictionary. Row is an integer place and v is a smaller dictionary.
        d: dict = json.loads(line)
        row += 1
        for _, v in d.items(): # we use the _ because we don't want the ID.
            if earlyStop > 0 and row >= earlyStop:
                # If we don't want the entire dataset, stop loading more than we want
                return tnsr, labels, weights
            if row % 100 == 0:
                print(f'Now loading event {row}/{length}')
            # append the label to our list
            labels.append(flares[v['label']])
            
            # Break each individual dictionary into dictionaries of observations
            # Key is the string in magDict, and timeDict is a dictionary of observations over time
            for key, timeDict in v['values'].items():
                # Turn our name string into a numeric value
                location = magDict[key]
                # Get the measurements out of the time series dictionary
                for timeStamp, measurement in timeDict.items():
                    tnsr[row][location][int(timeStamp)] = measurement
    print(f'{row} lines loaded.')
    # Close the file. I'm not a heathen                    
    file.close()
    # This might be a good place to perform some post processing, but that's a question for another day.
    # Famous last words.
    return tnsr, labels, weights



In [8]:
# This file has 77270 data points. 71633 do not have NAN values.
# %time train1, labels1, weights1 = getDataFromJSON(path="data/train_partition1_data.json", earlyStop=-1)
train1, labels1 = subSample("data/train_partition1_data.json", device=device)

Now loading event 1/785
Now loading event 101/785
Now loading event 201/785
Now loading event 301/785
Now loading event 401/785
Now loading event 501/785
Now loading event 601/785
Now loading event 701/785
785 lines loaded.


In [9]:
# This file has 93767 data points. 82425 of them do not have NAN values.  
# %time train2, labels2, weights2 = getDataFromJSON(path="data/train_partition2_data.json", earlyStop=-1)
train2, labels2 = subSample("data/train_partition2_data.json", device=device)

Now loading event 1/300
Now loading event 101/300
Now loading event 201/300
300 lines loaded.


In [10]:
# This file has 42986 data points. 37759 of them do not have NAN values.  
# %time train3, labels3, weights3 = getDataFromJSON(path="data/train_partition3_data.json", earlyStop=-1)
train3, labels3 = subSample("data/train_partition3_data.json", device=device)

Now loading event 1/585
Now loading event 101/585
Now loading event 201/585
Now loading event 301/585
Now loading event 401/585
Now loading event 501/585
585 lines loaded.


In [11]:
train2[0,0]

tensor([4318.6987, 4309.0054, 4307.6606, 4301.8848, 4288.6157, 4314.5142,
        4359.2119, 4366.2197, 4366.2666, 4393.8687, 4408.9658, 4407.9067,
        4431.2881, 4476.7900, 4461.4048, 4525.5620, 4545.3149, 4509.4556,
        4566.3525, 4539.2534, 4602.8027, 4594.4609, 4604.9976, 4609.1436,
        4598.2563, 4632.2480, 4619.8936, 4598.3965, 4615.3745, 4660.2656,
        4617.4736, 4595.8281, 4613.2896, 4588.9224, 4618.5352, 4571.0889,
        4555.6548, 4569.3770, 4567.5093, 4578.7480, 4631.6655, 4664.3589,
        4703.3726, 4676.2563, 4676.0078, 4704.5059, 4693.2700, 4714.9692,
        4821.5654, 4722.6094, 4659.8555, 4695.3633, 4758.1704, 4780.7847,
        4715.7080, 4751.1899, 4770.4849, 4760.3877, 4767.5723, 4788.1909])

In [12]:
# Define the network. Make sure to end with nn.Softmax activation
import torch.nn as nn
from skorch import NeuralNet

class logRegWithHidden(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, num_classes=5, drop1=.5, input_size=1980):
        super().__init__() 
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layerout = nn.Linear(hidden_size2, num_classes)
        #Define a RELU Activation unit
        self.relu = nn.ReLU()  
        self.smax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(p=drop1)

    def forward(self, x):
        #Forward Propagate through the layers as defined above
        y = self.drop(x.reshape(-1, 1980))
        y = self.drop(self.relu(self.layer1(y)))
        y = self.relu(self.layer2(y))
        y = self.smax(self.layerout(y))
        return y





In [64]:
def train(model, inputs, labels, weight, valSets, valLabels, valweight, lr=0.01):
    # TODO: Is this right? How do I determine the weights here?
    if weight is not None: weight = torch.Tensor(weight)
    lfc = nn.CrossEntropyLoss(weight=weight)
#     valLabels = torch.tensor(valLabels, dtype=torch.int)
    #ideas
    # 1-(weight/np.sum(weight))
    # .2/weight - this one normalizes so that each class is responsible for 20% of the loss
    # 1/weight - this is a bit naive, but the classes with fewer items are weighted more.
    # 1/(weight+1) - makes sure we don't have any pesky zeroes
    # np.sum(weight)/weight if your learning rate is too low.
    
    # Hyperparameters
    batch = 256
    epochs = 4 # artificially low for debugging
    
    # Start a dataloader object
    data = list(zip(inputs,labels))
    val = list(zip(valSets,valLabels))
    loader = DataLoader(data, batch_size = batch, num_workers=4)
    valLoader = DataLoader(val, batch_size = int(len(val)/4), num_workers=4)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    print(opt.state_dict())

    for epoch in range(epochs):
        model.train()
        batch_loss = []
        for (xtrain,ytrain) in loader:
            opt.zero_grad()
            output = model(xtrain)
            loss = lfc(output,ytrain)
            loss.backward()
            opt.step()
            batch_loss.append(loss.item())
        print(f'The training loss for epoch {epoch+1}/{epochs} was {np.mean(batch_loss)}')
        
        
        model.eval()
        balanced = [[],[],[],[],[]]
        batchLoss = []
        unbalanced = []
        
        for (xval,yval) in valLoader:
            output = model(xval)
            loss = lfc(output,yval)
            batchLoss.append(loss.item())
            corrects = yval.clone().detach() == torch.argmax(output)
            unbalanced.append(np.mean([1 if correct else 0 for correct in corrects.detach()]))
            
            for i, ans in enumerate(yval):
                balanced[ans].append(corrects[i])
        
        balanced = [np.mean(i) for i in balanced]
        balancedAccuracy = np.mean(balanced)
        
#         print(f'The total balanced accuracy for validation was {balancedAccuracy}')
        print(f'The unbalanced validation accuracy is {np.mean(unbalanced)}')
        print(f'The accuracy for each is {balanced}')           
        print(f'The validation loss was :   {epoch+1}/{epochs} was {np.mean(batchLoss)}')
    print(opt.state_dict())


    return model


In [67]:
model = logRegWithHidden(4096*4, 2048*4, drop1=0).to(device)
    # print(weights1+weights2, 1.0/np.array(weights1))

In [ ]:
print(model)
# [weights1[i] + weights2[i] for i in range(5)]
%time newModel = train(model, torch.cat((train1, train2), dim=0), labels1 + labels2, None, train3, labels3, None, lr = 0.0001)


logRegWithHidden(
  (layer1): Linear(in_features=1980, out_features=16384, bias=True)
  (layer2): Linear(in_features=16384, out_features=8192, bias=True)
  (layerout): Linear(in_features=8192, out_features=5, bias=True)
  (relu): ReLU()
  (smax): Softmax(dim=1)
  (drop): Dropout(p=0, inplace=False)
)
{'state': {}, 'param_groups': [{'lr': 0.0001, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'params': [0, 1, 2, 3, 4, 5]}]}
The training loss for epoch 1/10 was 1.6656454086303711
The unbalanced validation accuracy is 0.14794520547945206
The accuracy for each is [0.0, 0.0, 0.0, 0.47863247863247865, 0.4444444444444444]
The validation loss was :   1/10 was 1.751406764984131
The training loss for epoch 2/10 was 1.6656454086303711
The unbalanced validation accuracy is 0.14794520547945206
The accuracy for each is [0.0, 0.0, 0.0, 0.47863247863247865, 0.4444444444444444]
The validation loss was :   2/10 was 1.751406764984131
The training loss for epoch 3/10 was 1.66564

In [46]:
s = 1 # choose this to check the sth layer
feature_extraction1 = [child for child in model.children()]
print(feature_extraction1[s])
feature_extraction2 = [child for child in newModel.children()]
print(feature_extraction2[s])
print(torch.max(feature_extraction1[s].weight - feature_extraction2[s].weight).detach())
print(torch.min(feature_extraction1[s].weight - feature_extraction2[s].weight).detach())

Linear(in_features=4096, out_features=2048, bias=True)
Linear(in_features=4096, out_features=2048, bias=True)
tensor(0.)
tensor(0.)


In [59]:
_ = train(
    model,
    torch.cat((train1, train3), dim=0),
    labels1 + labels3,
    None,
    train2,
    labels2,
    None,
    lr = 10
)

The training loss for epoch 1/10 was 1.690267562866211
The unbalanced validation accuracy is 0.18666666666666668
The accuracy for each is [0.0, 0.0, 0.75, 0.0, 0.18333333333333332]
The validation loss was :   1/10 was 1.7014981508255005
The training loss for epoch 2/10 was 1.690267562866211
The unbalanced validation accuracy is 0.18666666666666668
The accuracy for each is [0.0, 0.0, 0.75, 0.0, 0.18333333333333332]
The validation loss was :   2/10 was 1.7014981508255005
The training loss for epoch 3/10 was 1.690267562866211
The unbalanced validation accuracy is 0.18666666666666668
The accuracy for each is [0.0, 0.0, 0.75, 0.0, 0.18333333333333332]
The validation loss was :   3/10 was 1.7014981508255005
The training loss for epoch 4/10 was 1.690267562866211
The unbalanced validation accuracy is 0.18666666666666668
The accuracy for each is [0.0, 0.0, 0.75, 0.0, 0.18333333333333332]
The validation loss was :   4/10 was 1.7014981508255005
The training loss for epoch 5/10 was 1.6902675628662

In [60]:
_ = train(
    model,
    torch.cat((train3, train2), dim=0),
    labels3 + labels2,
    None,
    train1,
    labels1,
    None,
    lr = 0.00000001
)

The training loss for epoch 1/10 was 1.7027375102043152
The unbalanced validation accuracy is 0.16122448979591836
The accuracy for each is [0.0, 0.0, 0.7770700636942676, 0.0, 0.22929936305732485]
The validation loss was :   1/10 was 1.733405590057373
The training loss for epoch 2/10 was 1.7027375102043152
The unbalanced validation accuracy is 0.16122448979591836
The accuracy for each is [0.0, 0.0, 0.7770700636942676, 0.0, 0.22929936305732485]
The validation loss was :   2/10 was 1.733405590057373
The training loss for epoch 3/10 was 1.7027375102043152
The unbalanced validation accuracy is 0.16122448979591836
The accuracy for each is [0.0, 0.0, 0.7770700636942676, 0.0, 0.22929936305732485]
The validation loss was :   3/10 was 1.733405590057373
The training loss for epoch 4/10 was 1.7027375102043152
The unbalanced validation accuracy is 0.16122448979591836
The accuracy for each is [0.0, 0.0, 0.7770700636942676, 0.0, 0.22929936305732485]
The validation loss was :   4/10 was 1.733405590057